# Exploring Immigration Trends
### Data Engineering Capstone Project

#### Summary
In this project we will be processing data to aid in the exploration of possible relationships between US immigration data, temperature data, and city demographic data. We will create an ETL pipeline to produce a dataset which can be loaded into a data warehouse and analyzed by data scientists.

The project has the following steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import re
import os
from datetime import datetime, timedelta
from functools import reduce
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, month, year, col, monotonically_increasing_id
from pyspark.sql.types import BooleanType

pd.set_option('display.max_columns', 200)

In [2]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
We will be combining immigration data with temperature and demographic data by city to create a data model for exploring where foreigners travel based on temperature and destination city demographics.  This could be used to target ads at the demogrpahics arriving at specific cities during certain times of year.

Some example queries would be 

 - "What city gets the most visitors in a particular month?"
 - "In which month does a particular city get the most tourists?"
 - "From where do most tourists visiting a specific city originate?"
 - "Top cities visited by foreigners from a specific country?"
 - "Difference in visitors during hottest and coldest month of a year"
 - "Average/median age of tourist visiting hot cities?"
 - "Average age of tourist visiting cold cities?"
 - "Top countries whose residents visit demographically youngest cities?"
 - "In what months are demographically older cities more popular with tourists?"

It is worth noting that the city of entry is not necessarily where the arriving person is visiting.  Tourists frequently travel to another destination within the US during their stay.  The immigration dataset tracks this with the `i94addr` field which denotes the state in which the respondent will reside during her stay.  However the temperature and city demographic datasets use a city field to denote location.  Therefore, for this analysis we are going to assume that most visitors are visiting the city in which they enter the USA and not travelling on to other destinations with significantly different weather.

#### Describe and Gather Data 

##### **Immigration Dataset**: [Link](https://travel.trade.gov/research/reports/i94/historical/2016.html)
We use 2016 I94 immigration data gathered by the US National Tourism and Trade Office to record entries into the US by foreigners via land, sea, or air.  This data has been provided in SAS7BDAT files as part of the project and a sample label description has been provided.  Some interesting features of the dataset include:

- ``i94yr``: 4 digit year of entry
- `i94mon`: numeric month of entry
- `i94cit`: 3 digit code for country of origin
- `i94port`: 3 digit code for port of entry
- `arrdate`: arrival date in USA (SAS date numeric field, time delta since 1/1/1960)
- `i94mode`: enumeration of entry type (1=air, 2=sea, 3=land, 9='not reported')
- `i94addr`: state of address during stay in the US
- `depdate`: date of departure from USA (SAS date numeric field, time delta since 1/1/1960)
- `i94bir`: age of respondent
- `i94visa`: enumeration representing reason for visit (1=business, 2=pleasure, 3=student)
- `biryear`: 4 digit birth year of respondent
- `gender`: gender of respondent

**Note:** We can use the provided `I94_SAS_Labels_Description` to retrieve the city of entry from the provided `i94port` code.  The matching city is what will be used to combine the immigration data with temperature and city demographic data.  The `i94port` codes do not match the codes for airports listed in the file `airport-codes_csv.csv`, so we will establish correspondence between datasets using the city name.

##### **Temperature Dataset**: [Link](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)
We are using montlhy averaged global temperature data downloaded from Kaggle and produced by Berkeley Earth.  The dataset has records from 1843-2013.  Below are the columns of the dataset:

- `dt`: date of measurement (YYYY-MM-DD)
- `AverageTemperature`: average temperature recorded
- `AverageTemperatureUncertainty`: average uncertainty of measurement
- `City`: city of measurement
- `Country`: country of measurement
- `Latitude`: geographic latitude
- `Longitude`: geographic longitude

##### **US City Demographics Dataset**: [Link](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
The city demographic data comes from OpenSoft, licensed under the public domain.   It contains demographics data from all US cities and census-designated places with a population greater or equal to 65,000.  It was produced by US Census Bureau's 2015 American Community Survey.  Some interesting attributes for our purposes are:

- `City`: city/municipality
- `State`: US state in which the city belongs
- `Median Age`: median age of resident (decimal)
- `Male Population`: count of male residents
- `Female Population`: count of female residents
- `Total Population`: count of all residents
- `Number of Veterans`: count of veteran residents
- `Foreign-born`: count of foreign-born residents
- `Average household size`: average number of residents per household
- `State Code`: state abbreviation
- `Race`: race identified by respondent
- `Count`: number of respondents identifying as reported race


### Step 2: Explore and Assess the Data
#### Explore the Data 
Here we view some data and identify quality issues, like missing values, duplicate data, etc.

In [2]:
# One month of immigration data
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immigration = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
df_immigration.head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,1.0,NaN,NaN,NaN,T,NaN,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,NaN,NaN,T,O,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [3]:
# Read world temperature data
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperatures = pd.read_csv(fname)
df_temperatures.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


##### Find range of temperature measurements and check consistency of using first of month as day of measurement

In [7]:
# convert time strings to datetime objects to find oldest and most recent measusrements
df_temperatures['dt'] = pd.to_datetime(df_temperatures['dt'])
print("Min date of measurement:", df_temperatures['dt'].min(axis=0))
print("Max date of measurement:", df_temperatures['dt'].max(axis=0))

#check if all measurements on the first of the month
print('Smallest day of month of measurement', df_temperatures['dt'].dt.day.min(axis=0))
print('Largest day of month of measurement', df_temperatures['dt'].dt.day.max(axis=0))

Min date of measurement: 1743-11-01 00:00:00
Max date of measurement: 2013-09-01 00:00:00
Smallest day of month of measurement 1
Largest day of month of measurement 1


In [8]:
fname = './us-cities-demographics.csv'
df_city_demos = pd.read_csv(fname, delimiter=';')
df_city_demos.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


##### Check rows for a specific city

In [10]:
df_city_demos[df_city_demos['City'] == 'San Francisco']

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
945,San Francisco,California,38.3,439752.0,425064.0,864816,26276.0,297199.0,2.37,CA,Hispanic or Latino,132114
2569,San Francisco,California,38.3,439752.0,425064.0,864816,26276.0,297199.0,2.37,CA,Asian,324034
2756,San Francisco,California,38.3,439752.0,425064.0,864816,26276.0,297199.0,2.37,CA,Black or African-American,53270
2805,San Francisco,California,38.3,439752.0,425064.0,864816,26276.0,297199.0,2.37,CA,White,442155
2838,San Francisco,California,38.3,439752.0,425064.0,864816,26276.0,297199.0,2.37,CA,American Indian and Alaska Native,8997


**Note:
As show above, the demographics data set is denormalized on race.  There is a count for every race in a city**

#### Cleaning Steps

We need to know the valid countires of origin and valid cities where people enter the US.  For this we create a lookup tables with valid values

In [40]:
def create_codes_to_cities(fpath):
    """
    Create a dict of valid i94 port codes to city
    
    Params:
    fpath: filepath to file of valid port codes to cities
    """
    codes_to_cities = {}
    with open(fpath, 'r') as file:
        for line in file:
            match = re.search(r"^\s+\'(.+)\'\s+=\s+\'(.+)\'\s+$", line)
            if match:
                s = match[2].rstrip().split(', ')
                if len(s) > 1:
                    state = s[-1][0:2].lower()
                    city = ','.join(s[0:-1]).lower()
                else:
                    s = s[0].rstrip().split(' ')
                    state = s[-1].lower()
                    city = ','.join(s[0:-1]).lower()
                codes_to_cities[match[1]] = { 'city': city, 'state': state }
    return codes_to_cities


def create_cities_to_codes(codes_to_cities):
    """
    Create a reverse index of the codes_to_cities dictionary
    
    Params:
    codes_to_cities: a dictionary with with port codes to cities
    """
    cities_to_codes = {}
    for k, v in codes_to_cities.items():
        cities_to_codes[v['city']] = k
    return cities_to_codes
    
    
codes_to_cities = create_codes_to_cities('./port_code_to_city.txt')
cities_to_codes = create_cities_to_codes(codes_to_cities)

@udf 
def city_to_i94port_code(city):
    """ Get i94port code from city """
    return cities_to_codes.get(city.lower(), None)

def create_codes_to_countries(fpath):
    """
    Create a dict of i94 codes to country
    
    Params:
    fpath: filepath to file of codes to countries
    """
    codes_to_countries = {}
    with open(fpath, 'r') as file:
        for line in file:
            match = re.search(r"\s+(\d{3})\s+=\s+\'(\w+)\'", line)
            if match:
                code = match[1]
                country = match[2].split('INVALID: ')[-1]
                codes_to_countries[int(code)] = country.lower()
    return codes_to_countries

codes_to_countries = create_codes_to_countries('./country_codes.txt')

@udf
def i94cit_code_to_country(code):
    """ convert code to country """
    country = codes_to_countries.get(int(code), None)
    if country:
        country = country.title()
    return country

In [4]:
def non_null_df(df, required_cols):
    """ Returns rows of dataframe where non-null values are in all required columns """
    return df.where(reduce(lambda x, y: x & y, (col(x).isNotNull() for x in required_cols)))


In [5]:
# UDF for checking i94port code validity
def valid_i94port(x):
    """ check if i94port is a valid code """
    return x in codes_to_cities

# UDF for checking i94cit code validity
def valid_i94cit(x):
    """ check if i94cit is a valid code """
    return x in codes_to_countries

i94port_filter = udf(valid_i94port, returnType=BooleanType())
i94cit_filter = udf(valid_i94cit, returnType=BooleanType())
    
def clean_immigration_data(df):
    """
    Remove rows with NaNs in required columns 
    or invalid port or invalid origin country
    """
    required_cols = ['i94yr', 'i94mon', 'i94cit', 'i94port', 'arrdate', 'i94mode',
                     'depdate', 'i94addr', 'gender', 'i94bir', 'i94visa']
    
    df = df.filter(i94port_filter(df.i94port) & i94cit_filter(df.i94cit))
    df = non_null_df(df, required_cols)
    return df

# Test
# fpath = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
# df_imms = spark.read.format('com.github.saurfang.sas.spark').load(fpath)
# df_imms = clean_immigration_data(df_imms)
# df_imms.toPandas().head(5)

In [6]:
def clean_temperature_data(df, min_year=None, max_year=None):
    """ 
    Remove rows with NaNs in required colsand filter for only US cities 
    Params:
    min_year: lowest year of data
    max_year: hightest year of data
    """
    required_cols = ['dt', 'AverageTemperature', 'City', 'Country', 'Latitude', 'Longitude']
    df = df.filter(col('Country') == 'United States')
    if min_year:
        df = df.filter(year('dt') >= min_year)
    if max_year:
        df = df.filter(year('dt') >= max_year)
    df = non_null_df(df, required_cols)
    return df

# Test
# fpath = '../../data2/GlobalLandTemperaturesByCity.csv'
# df_temps = spark.read.format('csv').option('header', 'true').load(fpath)
# df_temps = clean_temperature_data(df_temps)
# df_temps.toPandas().head(5)

In [21]:
# Retrive the max measurement date for all cities
df_max_measurement_date = (df_temps.withColumn('datetime', col('dt').cast('timestamp')).groupBy('City').agg(F.max('dt')))
df_max_measurement_date.toPandas()

# Retrive the min measurement
df_max_measurement_date.agg(F.min('max(dt)')).toPandas()

,min(max(dt))
0,2013-08-01


#### Test getting most recent month's temps for a specific city

In [26]:
# Test getting most recent months' measurements for San Francisco
df_temps_max_per_month = df_temps.groupBy(['City', month('dt')]).agg(F.max('dt'))\
                         .where(col('City') == 'San Francisco')
df_temps_max_per_month.toPandas()


,City,month(dt),max(dt)
0,San Francisco,9,2013-09-01
1,San Francisco,11,2012-11-01
2,San Francisco,12,2012-12-01
3,San Francisco,8,2013-08-01
4,San Francisco,10,2012-10-01
5,San Francisco,2,2013-02-01
6,San Francisco,1,2013-01-01
7,San Francisco,4,2013-04-01
8,San Francisco,6,2013-06-01
9,San Francisco,7,2013-07-01


In [30]:
ordered_df_temp = df_temps_max_per_month.alias('df1').join(df_temps.alias('df2'),
                                                           ((col('df1.max(dt)') == col('df2.dt'))
                                                           & (col('df1.City') == col('df2.City'))),
                                                           'inner') \
                                                     .drop(col('df2.City')) \
                                                     .orderBy(col('df1.City'), col('df1.month(dt)')).toPandas()
ordered_df_temp

,City,month(dt),max(dt),dt,AverageTemperature,AverageTemperatureUncertainty,Country,Latitude,Longitude
0,San Francisco,1,2013-01-01,2013-01-01,8.32,0.278,United States,37.78N,122.03W
1,San Francisco,2,2013-02-01,2013-02-01,10.229,0.494,United States,37.78N,122.03W
2,San Francisco,3,2013-03-01,2013-03-01,13.505999999999998,0.325,United States,37.78N,122.03W
3,San Francisco,4,2013-04-01,2013-04-01,15.995999999999999,0.419,United States,37.78N,122.03W
4,San Francisco,5,2013-05-01,2013-05-01,17.434,0.327,United States,37.78N,122.03W
5,San Francisco,6,2013-06-01,2013-06-01,19.759,0.33799999999999997,United States,37.78N,122.03W
6,San Francisco,7,2013-07-01,2013-07-01,20.656999999999996,0.36,United States,37.78N,122.03W
7,San Francisco,8,2013-08-01,2013-08-01,19.730999999999998,0.522,United States,37.78N,122.03W
8,San Francisco,9,2013-09-01,2013-09-01,20.471,0.826,United States,37.78N,122.03W
9,San Francisco,10,2012-10-01,2012-10-01,17.294999999999998,0.278,United States,37.78N,122.03W


In [7]:
def clean_city_demo_data(df):
    """ Remove rows with missing reqired data """
    required_cols = ['City', 'State', 'Median Age', 'Male Population',
                     'Female Population', 'Total Population', 'Number of Veterans',
                     'Foreign-born', 'Average Household Size', 'State Code', 'Race', 'Count']
    df = non_null_df(df, required_cols)
    return df

# Test
# fpath = './us-cities-demographics.csv'
# df_city_demos = spark.read.format('csv').option('header', 'true').option('delimiter', ';').load(fpath)
# df_city_demos = clean_city_demo_data(df_city_demos)
# df_city_demos.toPandas().head()

In [34]:
df_city_demos.withColumn('i94port_code', city_to_i94port_code(col('City'))).toPandas().head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count,i94port_code
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.6,MD,Hispanic or Latino,25924,None
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723,None
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759,None
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437,None
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402,None


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
We will be putting the data into a star schema.  The fact table will be immigration events into the US, combined with the temperature at the port of entry and some city demographic data.  We will have three dimension tables, one for immigration, one for tempearture, and one for city demographic data.  Most of our queries will revolve around temperature, age, gender trends, etc. so we are including dataset attributes that are related to these concepts.

Our data model assumes that the fact table and immigration table will appended over time and that the dimension tables for temperature and city demographics will be rebuilt when necessary to incorporate new data. This is important to remember as we will be putting the `i94port_code` into the rows of the temperature and city demographic dataframes to support joins between the fact and dimension tables.  We can expect temperature data to be relatively stable over time just like city demographic data.  If a new port is added to the valid port code list these dimension tables will need to be rebuilt, but this is expected to occur very infrequently.

##### Immigration fact table: 
Our fact table consists of immigration entry data plus the average monthly temperature for the month of entry.  Additionally we will include the median age and total city population data in the fact table in order to make aggregation quieries faster as we expect those types of queries to be most common.

##### Immigration dimension table:
The immigration dimension table just includes the relevant immigration data for each person entering the US.

##### Temperature dimension table:
In the cleanup phase we parred the data to only US city data for the most recent 12 months recorded.  This data is denormalized as there are 12 rows per city in the dataset.  Rather than normalize the data we are going to add the i94port code to the temperature data to allow joins with the fact table.

##### City Demographic dimension table:
The city demographic data is denormalized with respect to race.  So instead of splicing and creating a 'city_race' race join table with the a 'count' attribute, we are going to add the i94port code to the demographic rows to indicate the port.  This will allow for joins between the fact table and this city demographic dimension table.

#### 3.2 Mapping Out Data Pipelines
Steps necessary to pipeline the data into the chosen data model
1. Load the data into spark dataframes
1. Clean the dataframes by ensuring there are no NaNs for required columns
2. Scope the temperature data to contain only US cities and the most recent 12 months of measurement
3. Create the dimension tables for immigration, temperature, and city demographics, converting dates/times to a standard format
3. Drop cities from temperature and city demo dimension tables that dont have an i94port code
4. Run data quality checks after creation of each dimension table
4. Create the fact table for immigration event facts
5. Run data quality checks after creation of the fact table
5. Save the fact and dimension tables in parquet format for loading on HDFS compatible RDBMS like Redshift

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [8]:
@udf
def sas_daydelta_to_timestamp(sas_day_delta):
    """ Convert SAS day delta time since 1960 to datetime string """
    d = datetime.strptime('1960-01-01', '%Y-%m-%d') + timedelta(days=sas_day_delta)
    return d.strftime('%Y-%m-%d')

In [45]:
fpath = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
#fpath = '../../data/18-83510-I94-Data-2016/*.sas7bdat'
df_imms = spark.read.format('com.github.saurfang.sas.spark').load(fpath)
cleaned_imm_data = clean_immigration_data(df_imms)

dim_imm = cleaned_imm_data.select(col('i94yr').alias('year'),
                                  col('i94mon').alias('month'),
                                  col('i94cit').alias('origin_country_code'),
                                  col('i94port').alias('port_code'),
                                  sas_daydelta_to_timestamp(col('arrdate')).alias('arr_date'),
                                  sas_daydelta_to_timestamp(col('depdate')).alias('dep_date'),
                                  col('i94mode').alias('mode'),
                                  col('i94addr').alias('addr_state_abbr'),
                                  col('i94bir').alias('age'),
                                  col('i94visa').alias('reason'),
                                  col('biryear').alias('birth_year'),
                                  col('gender')
                                 )
dim_imm.toPandas().head(10)

,year,month,origin_country_code,port_code,arr_date,dep_date,mode,addr_state_abbr,age,reason,birth_year,gender
0,2016.0,4.0,101.0,WAS,2016-04-01,2016-08-25,1.0,MI,55.0,2.0,1961.0,M
1,2016.0,4.0,101.0,BOS,2016-04-01,2016-04-05,1.0,MA,58.0,1.0,1958.0,M
2,2016.0,4.0,101.0,ATL,2016-04-01,2016-04-05,1.0,MA,56.0,1.0,1960.0,F
3,2016.0,4.0,101.0,ATL,2016-04-01,2016-04-17,1.0,MA,62.0,2.0,1954.0,M
4,2016.0,4.0,101.0,ATL,2016-04-01,2016-05-04,1.0,NJ,49.0,2.0,1967.0,M
5,2016.0,4.0,101.0,ATL,2016-04-01,2016-06-06,1.0,NY,43.0,2.0,1973.0,M
6,2016.0,4.0,101.0,HOU,2016-04-01,2016-04-10,1.0,TX,53.0,2.0,1963.0,F
7,2016.0,4.0,101.0,NYC,2016-04-01,2016-04-17,1.0,NJ,37.0,2.0,1979.0,M
8,2016.0,4.0,101.0,NYC,2016-04-01,2016-04-23,1.0,NJ,49.0,2.0,1967.0,F
9,2016.0,4.0,101.0,NYC,2016-04-01,2016-05-01,1.0,NY,33.0,2.0,1983.0,M


In [31]:
fpath = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temps = spark.read.format('csv').option('header', 'true').load(fpath)
cleaned_temp_data = clean_temperature_data(df_temps)
# get most recent measurement per month per city
df_temps_max_per_month = cleaned_temp_data.groupBy(['City', month('dt')]).agg(F.max('dt'))
ordered_temp_data = df_temps_max_per_month.alias('df1').join(cleaned_temp_data.alias('df2'),
                                                             ((col('df1.max(dt)') == col('df2.dt'))
                                                             & (col('df1.City') == col('df2.City'))),
                                                             'inner') \
                                                       .drop(col('df2.City'))

dim_temp = ordered_temp_data.select(col('dt').alias('date'),
                                    col('AverageTemperature').alias('avg_temperature'),
                                    col('City').alias('city'),
                                    col('Country').alias('country'),
                                    col('Latitude').alias('latitude'),
                                    col('Longitude').alias('longitude'))\
                            .withColumn('port_code', city_to_i94port_code(col('city')))\
                            .dropna(subset=['port_code'])
                                    
dim_temp.toPandas().head(10)

,date,avg_temperature,city,country,latitude,longitude,port_code
0,2013-03-01,4.766,Seattle,United States,47.42N,121.97W,SEA
1,2013-05-01,19.159,Louisville,United States,37.78N,85.42W,LOU
2,2012-11-01,13.367,Sacramento,United States,37.78N,122.03W,SAC
3,2013-01-01,0.98,Tacoma,United States,47.42N,121.97W,TAC
4,2013-02-01,10.229,San Jose,United States,37.78N,122.03W,SNJ
5,2013-05-01,20.62,Birmingham,United States,32.95N,87.13W,BHX
6,2013-05-01,24.09,Tampa,United States,28.13N,82.73W,TAM
7,2013-06-01,23.418000000000003,Albuquerque,United States,34.56N,107.03W,ABQ
8,2013-07-01,25.678,Columbus,United States,32.95N,85.21W,CLM
9,2013-07-01,23.558000000000003,Columbus,United States,39.38N,83.24W,CLM


In [27]:
fpath = './us-cities-demographics.csv'
df_city_demos = spark.read.format('csv').option('header', 'true').option('delimiter', ';').load(fpath)
cleaned_city_demo_data = clean_city_demo_data(df_city_demos)
dim_city_demo = cleaned_city_demo_data.select(col('City').alias('city'),
                                              col('State').alias('state'),
                                              col('Median Age').alias('median_age'),
                                              col('Male Population').alias('male_population'),
                                              col('Female Population').alias('female_population'),
                                              col('Total Population').alias('total_population'),
                                              col('Number of Veterans').alias('veteran_population'),
                                              col('Foreign-born').alias('foreign_born_population'),
                                              col('Average Household Size').alias('avg_household_size'),
                                              col('State Code').alias('state_abbr'),
                                              col('Race').alias('race'),
                                              col('Count').alias('race_count'))\
                                      .withColumn('port_code', city_to_i94port_code(col('city')))\
                                      .dropna(subset=['port_code'])
dim_city_demo.toPandas().head(10)

,city,state,median_age,male_population,female_population,total_population,veteran_population,foreign_born_population,avg_household_size,state_abbr,race,race_count,port_code
0,Philadelphia,Pennsylvania,34.1,741270,826172,1567442,61995,205339,2.61,PA,Asian,122721,PHI
1,Fort Myers,Florida,37.3,36850,37165,74015,4312,15365,2.45,FL,White,50169,FMY
2,Laredo,Texas,28.8,124305,131484,255789,4921,68427,3.66,TX,American Indian and Alaska Native,1253,LAR
3,New Haven,Connecticut,29.9,63765,66545,130310,2567,25871,2.48,CT,American Indian and Alaska Native,2205,NWH
4,Salt Lake City,Utah,32.1,98364,94296,192660,6829,32166,2.38,UT,Asian,13153,SLC
5,Los Angeles,California,35.0,1958998,2012898,3971896,85417,1485425,2.86,CA,White,2177650,LOS
6,Tulsa,Oklahoma,35.0,197437,205654,403091,24672,43751,2.37,OK,Black or African-American,72643,TUL
7,Seattle,Washington,35.5,345659,338784,684443,29364,119840,2.13,WA,Hispanic or Latino,43060,SEA
8,Camden,New Jersey,27.9,36437,39694,76131,1425,11317,3.0,NJ,White,19907,CNJ
9,Omaha,Nebraska,34.2,218789,225098,443887,24503,48263,2.47,NE,Hispanic or Latino,63516,OMA


In [46]:
# join the dimension tables to create the fact table and select the columns for the fact table
# we use distinct() to remove duplicate entries which will occur from the denormalized race data
# in the city demographic dimension data being joined with the other tables
fact_imm = dim_imm.join(dim_temp, (dim_imm.port_code == dim_temp.port_code) & (dim_imm.month == month(dim_temp.date)))\
             .join(dim_city_demo, (dim_imm.port_code == dim_city_demo.port_code))\
             .select('year',
                     'month',
                     i94cit_code_to_country(col('origin_country_code')).alias('origin_country'),
                     dim_imm.port_code,
                     'arr_date',
                     'dep_date',
                     'mode',
                     'addr_state_abbr',
                     'age',
                     'reason',
                     'gender',
                     col('avg_temperature').alias('temperature_avg'),
                     col('median_age').alias('city_median_age'),
                     col('total_population').alias('city_total_population')).distinct()
fact_imm.toPandas().head(10)

,year,month,origin_country,port_code,arr_date,dep_date,mode,addr_state_abbr,age,reason,gender,temperature_avg,city_median_age,city_total_population
0,2016.0,4.0,Romania,DAL,2016-04-01,2016-07-06,1.0,TX,63.0,2.0,M,16.259,32.6,1300082
1,2016.0,4.0,India,DAL,2016-04-01,2016-09-02,1.0,TX,47.0,2.0,F,16.259,32.6,1300082
2,2016.0,4.0,Brazil,DAL,2016-04-01,2016-04-07,1.0,CA,42.0,1.0,F,16.259,32.6,1300082
3,2016.0,4.0,Israel,DAL,2016-04-02,2016-04-24,1.0,TX,68.0,2.0,M,16.259,32.6,1300082
4,2016.0,4.0,Australia,DAL,2016-04-02,2016-04-24,1.0,PA,43.0,2.0,F,16.259,32.6,1300082
5,2016.0,4.0,France,DAL,2016-04-03,2016-04-14,1.0,TX,39.0,1.0,M,16.259,32.6,1300082
6,2016.0,4.0,Italy,DAL,2016-04-03,2016-04-13,1.0,CA,39.0,1.0,M,16.259,32.6,1300082
7,2016.0,4.0,Indonesia,DAL,2016-04-03,2016-05-16,1.0,MO,22.0,3.0,M,16.259,32.6,1300082
8,2016.0,4.0,India,DAL,2016-04-03,2016-04-14,1.0,KS,39.0,1.0,M,16.259,32.6,1300082
9,2016.0,4.0,Thailand,DAL,2016-04-03,2016-04-09,1.0,TX,54.0,1.0,F,16.259,32.6,1300082


#### 4.2 Data Quality Checks

We will perform data quality check to ensure our fact and dim tables are not empyt.  We will also check that our renaming of columns has run successfully.  In addition because we did some aggregation on the temperature data to get the most recent 12 months for each city we will checkf that a city has 12 months of avg montly temperatures.

In [47]:
def has_rows(df, name):
    """
    Check if there's at least one row in a dataframe
    
    Params:
    df: spark dataframe
    name: friendly printable dataframe name
    """
    if df.count() == 0:
        print(f"'has_rows' validation failed. DataFrame {name} has 0 rows")
        return False
    return True

dfs = {
    'fact_immigrations': fact_imm,
    'dim_immigrations': dim_imm,
    'dim_temperatures': dim_temp,
    'dim_city_demos': dim_city_demo
}

if not all([has_rows(df, name) for name, df in dfs.items()]):
    raise Exception('has_rows checks failed')
else:
    print("'has_rows' checks passed")

'has_rows' checks passed


In [52]:
def has_required_cols(df, required_cols, name):
    """
    Check if dataframe has all the required columns
    
    Params:
    df: spark dataframe
    required_cols: list with required columns in desired order
    name: friendly printable dataframe name
    """
    if len(df.columns) != len(required_cols):
        print(f"'has_required_cols' validation failed. Number of cols do not match for DataFrame {name}")
    if not all([df.columns[i] == required_cols[i] for i in range(len(df.columns))]):
        print(f"'has_required_cols' validation failed.  DataFrame {name} has columns: {df.columns}"
              f"but expected {required_cols}")
        return False
    return True

required_cols_per_table = [
    {
        'name': 'fact_immigrations',
        'dataframe': fact_imm,
        'required_cols': ['year', 'month', 'origin_country', 'port_code', 'arr_date', 'dep_date', 'mode',
                          'addr_state_abbr', 'age', 'reason', 'gender', 'temperature_avg', 'city_median_age',
                          'city_total_population']
    },
    {
        'name': 'dim_immigrations',
        'dataframe': dim_imm,
        'required_cols': ['year', 'month', 'origin_country_code', 'port_code', 'arr_date', 'dep_date', 'mode',
                          'addr_state_abbr', 'age', 'reason', 'birth_year', 'gender'],
    },
    {
        'name': 'dim_temperatures',
        'dataframe': dim_temp,
        'required_cols': ['date', 'avg_temperature', 'city', 'country', 'latitude', 'longitude', 'port_code']
    },
    {
        'name': 'dim_city_demos',
        'dataframe': dim_city_demo,
        'required_cols': ['city', 'state', 'median_age', 'male_population', 'female_population', 'total_population',
                          'veteran_population', 'foreign_born_population', 'avg_household_size', 'state_abbr',
                          'race', 'race_count', 'port_code']
    }
]
if not all([has_required_cols(r['dataframe'], r['required_cols'], r['name']) for r in required_cols_per_table]):
    raise Exception('has_required_cols checks failed')
else:
    print("'has_required_cols' checks passed")

'has_required_cols' checks passed


In [53]:
# Simple sanity check for temperature df manipulations
def check_12_temps_per_city(dim_temp, city):
    """ Make sure there are only 12 temps for a city"""
    if dim_temp.select(dim_temp.columns).where(dim_temp.city == city).count() == 12:
        return True
    return False
city = 'San Francisco'
if not check_12_temps_per_city(dim_temp, city):
    raise Exception(f"'check_12_temps_per_city' validation failed for city '{city}'")
else:
    print("'check_12_temps_per_city' passed")

'check_12_temps_per_city' passed


#### 4.3 Save data for later load
Save the data to parquet format for loading into HDFS-based RDBMS

In [58]:
# save for later loads
output_data = './output_data/'
# can make mode 'append' for dim_imm and fact_imm once deployed in production
dim_imm.write.mode('overwrite').partitionBy('year', 'month').parquet(os.path.join(output_data, 'dim_immigrations'))
dim_temp.write.mode('overwrite').partitionBy('port_code').parquet(os.path.join(output_data, 'dim_temperatures'))
dim_city_demo.write.mode('overwrite').partitionBy('port_code').parquet(os.path.join(output_data, 'dim_city_demographics'))
fact_imm.write.mode('overwrite').partitionBy('year', 'month').parquet(os.path.join(output_data, 'fact_immigrations'))

#### 4.4 Data dictionary 
The attribute names of the various dataset have been renamed for consistency to the end user

##### Facts table:
- `year`: 4 digit year of entry
- `month`: numeric month of entry
- `origin_country`: country of origin
- `port_code`: 3 digit code for port of entry
- `arr_date`: arrival date in USA (SAS date numeric field, time delta since 1/1/1960)
- `dep_date`: departure date from USA (SAS date numeric field, time delta since 1/1/1960)
- `mode`: enumeration of entry type (1=air, 2=sea, 3=land, 9='not reported')
- `addr_state_abbr`: state of address during stay in the US
- `age`: age of respondent
- `reason`: enumeration representing reason for visit (1=business, 2=pleasure, 3=student)
- `gender`: gender of respondent
- `temperature_avg`: average temperature of the port city
- `city_median_age`: median age port city
- `city_total_population`: total population of port city

##### Immigration dimension table:
- `year`: 4 digit year of entry
- `month`: numeric month of entry
- `origin_country_code`: 3 digit code for country of origin
- `port_code`: 3 digit code for port of entry
- `arr_date`: arrival date in USA (SAS date numeric field, time delta since 1/1/1960)
- `dep_date`: departure date from USA (SAS date numeric field, time delta since 1/1/1960)
- `mode`: enumeration of entry type (1=air, 2=sea, 3=land, 9='not reported')
- `addr_state_abbr`: state of address during stay in the US
- `age`: age of respondent
- `reason`: enumeration representing reason for visit (1=business, 2=pleasure, 3=student)
- `birth_year`: 4 digit birth year of respondent
- `gender`: gender of respondent

##### Temperature dimension table:
- `date`: date of measurement (YYYY-MM-DD)
- `avg_temperature`: average temperature recorded
- `city`: city of measurement
- `country`: country of measurement
- `latitude`: geographic latitude
- `longitude`: geographic longitude
- `port_code`: 3 digit code for port of entry

##### City Demographic dimension table:
- `city`: city/municipality
- `state`: US state in which the city belongs
- `median_age`: median age of resident (decimal)
- `male_population`: count of male residents
- `female_population`: count of female residents
- `total_population`: count of all residents
- `veteran_population`: count of all veterans
- `foreign_born_population`: count of all foreign born residents
- `avg_household_size`: average number of residents per household
- `state_abbr`: state abbreviation
- `race`: race identified by respondent
- `race_count`: number of respondents identifying as reported race
- `port_code`: 3 digit code for port of entry


#### Step 5: Complete Project Write Up


__Clearly state the rationale for the choice of tools and technologies for the project:__

*    PySpark was used for the ease of exploring data.  It supports both functional and SQL style queries, and data can easily be parallelized by spreading the dataframes over several workers in a spark cluster

Write a description of how you would approach the problem differently under the following scenarios:

__The data was increased by 100x:__
   
* If the data were increased 100x then we would scale up the number of workers and parallelize processing.
        
__The data populates a dashboard that must be updated on a daily basis by 7am every day:__

*    We could run an Airflow DAG to process new data on a schedule and load it to the destination before 7am.  We could use SLAs and alarms to make sure the data is updated in time.
        
        
__The database needed to be accessed by 100+ people:__

*    If the resulting data model needs to be accessed by 100+ people we should consider loding the data in Redshift (ie. warehousing it) so that we get the benefits of parallelized queries.
        
        